In [2]:
%matplotlib notebook
#import matplotlib.pyplot as plt
# Note! ITK interacts weirdly here.  from lazy_imports import itk does not work.
# Additionally, import itk must occur before lazy_imports for itkwidgets.view (ie itkview) to work.
import itk
import vtk
#from lazy_imports import itk
from lazy_imports import np
from lazy_imports import plt
from lazy_imports import sitk
from lazy_imports import loadmat, savemat
from lazy_imports import sio
from lazy_imports import itkwidgets
from lazy_imports import itkview
from lazy_imports import interactive
from lazy_imports import ipywidgets
from lazy_imports import pv
from lazy_imports import torch
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)

In [3]:
from disp.vis import show_2d, show_2d_tensors
from disp.vis import vis_tensors, vis_path
from disp.vis import view_3d_tensors, tensors_to_mesh, view_3d_paths, path_to_tube

In [4]:
from data.io import readRaw, ReadScalars, ReadTensors, WriteTensorNPArray, WriteScalarNPArray, readPath3D
from data.convert import GetNPArrayFromSITK, GetSITKImageFromNP
import algo.metricModSolver as mms
import algo.geodesic as geo
import algo.euler as euler
import util.tensors as tensors
import util.diffeo as diffeo
import pickle
import math
import ipywebrtc
from IPython.display import display
import time

In [5]:
from dipy.io.image import load_nifti, save_nifti
from dipy.align.imaffine import AffineMap

In [6]:
# from colorbrewer2, sequential 9 values pasted together YlGnBu (reverse order) then YlOrRd
# EXCEPT THESE ARE NOT PRINT FRIENDLY OR PHOTOCOPY SAFE!!!
# But skip the yellows in the middle -- too light
geo_colors = ['tab:red', 'tab:pink', 'tab:orange', 'tab:blue', 'tab:purple', 'tab:green', 'tab:cyan']
eul_colors = ['k', 'tab:gray', 'tab:brown', 'm', 'y', 'tab:olive', 'maroon']
#interp_colors = ['#081d58', '#253494', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4', '#edf8b1', '#ffffd9',
#                 '#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026']
interp_colors = ['#081d58', '#253494', '#225ea8', '#1d91c0', '#41b6c4', '#7fcdbb', '#c7e9b4',
                 '#fed976', '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026']

blue=interp_colors[3]
ltblue=interp_colors[6]
yellow=interp_colors[7]
orange=interp_colors[9]
red=interp_colors[12]

# 3 classes are safest, but fewer options.
# Stitch together some options from sequential 3 values YlGnBu (reverse order), YlOrBr and 
#interp_colors = ['#2c7fb8', '#7fcdbb', '#edf8b1', '#fff7bc', '#fec44f', '#d95f0e']

# Compromise, use 5 class YlGnBu (reverse order) and YlOrBr again leaving out yellow
interp_colors = ['#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#fed98e', '#fe9929', '#d95f0e', '#993404']
# leave out #ffffcc and #ffffd4

blue=interp_colors[1]
ltblue=interp_colors[2]
yellow=interp_colors[4]
orange=interp_colors[5]
red=interp_colors[7]

# Same as above but with a dark gray added in the front
interp_colors = ['#777777','#253494', '#2c7fb8', '#41b6c4', '#a1dab4', '#fed98e', '#fe9929', '#d95f0e', '#993404']

blue=interp_colors[2]
ltblue=interp_colors[3]
yellow=interp_colors[5]
orange=interp_colors[6]
red=interp_colors[8]

# matches defaults
XSMALL_SIZE = 12
SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20
# bigger for papers
XSMALL_SIZE = 12
SMALL_SIZE = 17
MEDIUM_SIZE = 23
BIGGER_SIZE = 25

plt.rcParams["figure.figsize"] = (4, 4) # (w, h)
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title



# Look at DTI-TK Results

In [7]:
def convert_to_itk(tfm, invert=False):
  #print('converting transform to ITK format', tfm_prefix + '.txt')
  print('converting transform to ITK format')
  #tfm = np.loadtxt(tfm_prefix + '.txt')
  stfm = sitk.AffineTransform(3)
  stfm.SetMatrix(tuple(tfm[0:3,0:3].flatten()))
  stfm.SetTranslation(tuple(tfm[:3,3]))
  #sitk.WriteTransform(stfm, tfm_prefix + '_itk.txt')
  if invert:
    return(stfm.GetInverse())
  else:
    return(stfm)    
                      
def transform_tensors(tens, tfm, stfm):
  print('tens shape:',tens.shape)
  #tfm = np.loadtxt(tfm_prefix+'.txt')
  #stfm = sitk.ReadTransform(tfm_prefix+'_itk.txt')

  xsz = tens.shape[0]
  ysz = tens.shape[1]
  zsz = tens.shape[2]

  resamp_tens = np.zeros_like(tens)
  for cc in range(6):
    resamp_tens[:,:,:,cc] = GetNPArrayFromSITK(sitk.Resample(GetSITKImageFromNP(tens[:,:,:,cc].astype(np.float32)), stfm))
  
  tens_full = np.zeros((xsz,ysz,zsz,3,3))
  tens_full[:,:,:,0,0] = resamp_tens[:,:,:,0]
  tens_full[:,:,:,0,1] = resamp_tens[:,:,:,1]
  tens_full[:,:,:,0,2] = resamp_tens[:,:,:,2]
  tens_full[:,:,:,1,0] = tens_full[:,:,:,0,1]
  tens_full[:,:,:,2,0] = tens_full[:,:,:,0,2]
  tens_full[:,:,:,1,1] = resamp_tens[:,:,:,3]
  tens_full[:,:,:,1,2] = resamp_tens[:,:,:,4]
  tens_full[:,:,:,2,1] = tens_full[:,:,:,1,2]
  tens_full[:,:,:,2,2] = resamp_tens[:,:,:,5]

  
  out_tens = np.einsum('...ij,...jk,...lk->...il',tfm[:3,:3],tens_full,tfm[:3,:3])

  out_tens_tri = np.zeros((xsz,ysz,zsz,6))
  out_tens_tri[:,:,:,0] = out_tens[:,:,:,0,0]
  out_tens_tri[:,:,:,1] = out_tens[:,:,:,0,1]
  out_tens_tri[:,:,:,2] = out_tens[:,:,:,0,2]
  out_tens_tri[:,:,:,3] = out_tens[:,:,:,1,1]
  out_tens_tri[:,:,:,4] = out_tens[:,:,:,1,2]
  out_tens_tri[:,:,:,5] = out_tens[:,:,:,2,2]

  print('out tens tri shape:',out_tens_tri.shape)
  return(out_tens_tri)


#convert_to_itk(tfm_pref)
#out_dti = transform_tensors(in_dti, tfm_pref)

In [16]:
outroot = '/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/UKF_experiments/BallResults/'

identity = np.eye(4)

#atlas_tens, atlas_aff = load_nifti(f"{outroot}DTITKReg/104416_resampled_aff_reg_to_108222_diffeo.nii.gz")
atlas_tens, atlas_aff = load_nifti(f"{outroot}DTITKReg/104416_padded_aff_reg_to_108222_diffeo.nii.gz")
atlas_tens, atlas_aff = load_nifti(f"{outroot}DTITKReg/mean_diffeomorphic_initial6.nii.gz")
atlas_tens = atlas_tens.squeeze()
#aff_map = AffineMap(identity, atlas_tens.shape, atlas_aff)
#atlas_tens_tfm = aff_map.transform(atlas_tens)
atlas_aff_itk = convert_to_itk(atlas_aff,True)
atlas_tens_tfm = transform_tensors(atlas_tens, atlas_aff, atlas_aff_itk)

#atlas_mask = ReadScalars(f"{outroot}DTITKReg/108222_resampled_tr_mask.nii.gz")
atlas_mask = ReadScalars(f"{outroot}DTITKReg/108222_padded_tr_mask.nii.gz")
atlas_mask = ReadScalars(f"{outroot}DTITKReg/mean_affine3_tr_mask.nii.gz")
atlas_mask, aff = load_nifti(f"{outroot}DTITKReg/mean_affine3_tr_mask.nii.gz")
atlas_mask = atlas_mask.squeeze()

#atlas_tens_orig_dims, atlas_aff = load_nifti(f"{outroot}DTITKReg/104416_resampled_aff_reg_to_108222_diffeo_orig_dims.nii.gz")
atlas_tens_orig_dims, atlas_aff = load_nifti(f"{outroot}DTITKReg/104416_padded_aff_reg_to_108222_diffeo_orig_dims.nii.gz")
atlas_tens_orig_dims, atlas_aff = load_nifti(f"{outroot}DTITKReg/mean_diffeomorphic_initial6_orig_dims.nii.gz")
atlas_tens_orig_dims = atlas_tens_orig_dims.squeeze()
#aff_map = AffineMap(identity, atlas_tens_orig_dims.shape, atlas_aff)
#atlas_tens_orig_dims_tfm = aff_map.transform(atlas_tens_orig_dims)
atlas_aff_itk = convert_to_itk(atlas_aff,True)
atlas_tens_orig_dims_tfm = transform_tensors(atlas_tens_orig_dims, atlas_aff, atlas_aff_itk)


converting transform to ITK format
tens shape: (256, 256, 256, 6)
out tens tri shape: (256, 256, 256, 6)
converting transform to ITK format
tens shape: (145, 174, 145, 6)
out tens tri shape: (145, 174, 145, 6)


In [22]:
print(atlas_aff,atlas_aff_itk)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] itk::simple::Transform
 AffineTransform (0x55c5a64b9960)
   RTTI typeinfo:   itk::AffineTransform<double, 3u>
   Reference Count: 1
   Modified Time: 3310
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     1 0 0 
     0 1 0 
     0 0 1 
   Offset: [-0, -0, -0]
   Center: [0, 0, 0]
   Translation: [0, 0, 0]
   Inverse: 
     1 0 0 
     0 1 0 
     0 0 1 
   Singular: 0



In [21]:
cases = []
cases.append('105923')
cases.append('100206')
cases.append('102715')
cases.append('104416')
cases.append('107422')
cases.append('108222')
t1s = []
in_tensors = []
out_tensors = []
outdirs = []
dtitkdirs = []

for run_case in cases:
  subj = run_case[:6]
  indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_UKF_data_with_grad_dev/' + subj + '/'
  outdir = f'{outroot}{run_case}_'
  outdirs.append(outdir)
  dtitkdir = f'{outroot}DTITKReg/{run_case}_'
  dtitkdirs.append(dtitkdir)

  #tens_file = 'scaled_orig_tensors_rreg_v2.nhdr'
  tens_file = 'scaled_orig_tensors_rreg.nhdr'
  in_tens = ReadTensors(outdir + tens_file)
  in_tensors.append(in_tens)
  #tens_file = 'resampled.nii.gz'
  tens_file = 'padded.nii.gz'
  tens_file = 'padded_aff_diffeo_orig_dims.nii.gz'
  out_tens, aff = load_nifti(dtitkdir + tens_file)
  out_tens = out_tens.squeeze()
  out_tensors.append(out_tens)
  ##aff_map = AffineMap(identity, out_tens.shape, aff)
  ##out_tens_tfm = aff_map.transform(out_tens)
  #aff_itk = convert_to_itk(aff,False)
  #out_tens_tfm = transform_tensors(out_tens, aff, aff_itk)
  #out_tensors.append(out_tens_tfm)


In [14]:
diff_tens_orig_dims, aff = load_nifti(f"{outroot}DTITKReg/104416_diffeo_orig_dims.nii.gz")
diff_tens_orig_dims = diff_tens_orig_dims.squeeze()
#aff_map = AffineMap(identity, atlas_tens_orig_dims.shape, atlas_aff)
#atlas_tens_orig_dims_tfm = aff_map.transform(atlas_tens_orig_dims)
aff_itk = convert_to_itk(aff,False)
diff_tens_orig_dims_tfm = transform_tensors(diff_tens_orig_dims, aff, aff_itk)


converting transform to ITK format
tens shape: (145, 174, 145, 6)
out tens tri shape: (145, 174, 145, 6)


In [36]:
aff_tens_orig_dims, aff = load_nifti(f"{outroot}DTITKReg/104416_padded_aff_reg_to_108222.nii.gz")
aff_tens_orig_dims = aff_tens_orig_dims.squeeze()
#aff_map = AffineMap(identity, atlas_tens_orig_dims.shape, atlas_aff)
#atlas_tens_orig_dims_tfm = aff_map.transform(atlas_tens_orig_dims)
aff_itk = convert_to_itk(aff,False)
aff_tens_orig_dims_tfm = transform_tensors(aff_tens_orig_dims, aff, aff_itk)


converting transform to ITK format
tens shape: (256, 256, 256, 6)
out tens tri shape: (256, 256, 256, 6)


In [17]:
itkview(atlas_mask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pr…

In [18]:
itkview(out_tensors[0][:,:,:,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [19]:
#atlas_tens = atlas_tens.squeeze()
#print(atlas_tens.shape,atlas_mask.shape,atlas_tens_orig_dims.shape)
itkview(atlas_tens[:,:,:,0])#, label_image=atlas_mask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [22]:
itkview(out_tensors[1][:,:,:,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [15]:
itkview(in_tensors[1][:,:,:,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [25]:
#tr = ReadScalars(f"{outroot}DTITKReg/108222_resampled_tr.nii.gz")
#tr = ReadScalars(f"{outroot}DTITKReg/108222_padded_tr.nii.gz")
tr = ReadScalars(f"{outroot}DTITKReg/mean_affine3_tr.nii.gz")

In [47]:
itkview(tr)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [26]:
itkview(tr)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [51]:
import dipy.reconst.dti as dti
from dipy.data import get_sphere
from dipy.viz import window, actor
sphere = get_sphere('repulsion724')

In [50]:
atlas_tens_full = np.zeros((atlas_tens.shape[0],atlas_tens.shape[1],atlas_tens.shape[2],3,3))
atlas_tens_full[:,:,:,0,0] = atlas_tens[:,:,:,0]
atlas_tens_full[:,:,:,0,1] = atlas_tens[:,:,:,1]
atlas_tens_full[:,:,:,0,2] = atlas_tens[:,:,:,2]
atlas_tens_full[:,:,:,1,0] = atlas_tens[:,:,:,1]
atlas_tens_full[:,:,:,1,1] = atlas_tens[:,:,:,3]
atlas_tens_full[:,:,:,1,2] = atlas_tens[:,:,:,4]
atlas_tens_full[:,:,:,2,0] = atlas_tens[:,:,:,2]
atlas_tens_full[:,:,:,2,1] = atlas_tens[:,:,:,4]
atlas_tens_full[:,:,:,2,2] = atlas_tens[:,:,:,5]
evals, evecs = np.linalg.eigh(atlas_tens_full)


In [53]:
FA = dti.fractional_anisotropy(evals)
FA[np.isnan(FA)] = 0
FA = np.clip(FA,0,1)
RGB = dti.color_fa(FA, evecs)


In [ ]:
scene = window.Scene()
xn=109
xx=139
yn=153
yx=172
zn=116
zx=124

evals_win = evals[xn:xx,yn:yx,zn:zx]
evecs_win = evecs[xn:xx,yn:yx,zn:zx]
cfa = RGB[xn:xx,yn:yx,zn:zx]
cfa /= cfa.max()
scene.add(actor.tensor_slicer(evals_win, evecs_win, scalar_colors=cfa, sphere=sphere,scale=0.3))
window.show(scene)